In [1]:
import pickle as pkl
import numpy as np
import random
from mxnet import nd
import mxnet as mx

with open("sequences.pkl", 'rb') as file:
    sequences = pkl.load(file)




In [2]:
from models.cluster import fit_pca, compute_pca_features
from time import time

train_seq_numb = int(len(sequences) * 0.7)
print(train_seq_numb)
train_seq = []
for sequence in sequences[: train_seq_numb]:
    for single_feature in sequence.person_features.values():
        train_seq.append(single_feature.copyto(mx.cpu()).asnumpy())
pca_train_data = np.concatenate(train_seq, axis=0)

all_seq = []
for sequence in sequences:
    for single_feature in sequence.person_features.values():
        all_seq.append(single_feature.copyto(mx.cpu()).asnumpy())
pca_transform_data = np.concatenate(all_seq, axis=0)

start = time()
print(pca_train_data.shape)
pca = fit_pca(256, pca_train_data)
print('training took', time() - start)
transformed_features = compute_pca_features(pca_transform_data, pca)



1184
(13701, 1024)
training took 0.8527817726135254
Computing PCA!
input shape: (19632, 1024) -----> (19632, 256)


In [3]:
from models.cluster import fit_kmeans

kmeans = fit_kmeans(10, transformed_features)

kmeans fitted!


In [4]:
from dataset.sequence import ClusteredSequence
from tqdm import tqdm

assigned_features = 0
clustered_sequences = []
for sequence in tqdm(sequences):
    person_id_numb = len(sequence.person_features)
    features_as_array = pca_transform_data[assigned_features: assigned_features + person_id_numb]
    pca_features = transformed_features[assigned_features: assigned_features + person_id_numb]
    cluster_labels = kmeans.labels_[assigned_features: assigned_features + person_id_numb]
    clustered_sequences.append(
        ClusteredSequence(gt=sequence.gt, name_pattern=sequence.name_pattern, klass=sequence.klass,
                          person_features=sequence.person_features,
                          cluster_labels=cluster_labels, features_as_array=features_as_array,
                          reduced_features=pca_features))
    assigned_features += person_id_numb
with open('clustered_sequences.pkl', 'wb') as file:
    pkl.dump(clustered_sequences, file)

assert assigned_features == pca_transform_data.shape[0]
assert assigned_features == transformed_features.shape[0]
assert assigned_features == kmeans.labels_.shape[0]



100%|██████████| 1692/1692 [00:00<00:00, 277444.87it/s]


In [5]:
with open('clustered_sequences.pkl', 'rb') as file:
    clustered_sequences = pkl.load(file)
len(clustered_sequences)

1692

In [15]:
from utils.color_generator import ColorGenerator
import gluoncv
import cv2

seq = clustered_sequences[4]
cg = ColorGenerator()
first_frame = seq.gt['frame'].min()
last_frame = seq.gt['frame'].max()

for frame in tqdm(range(first_frame, last_frame)):
    img = cv2.imread(seq.name_pattern.format(frame=frame))

    labels = seq.gt[seq.gt['frame'] == frame].to_records()
    bboxes = np.array(
        [[label['x1'], label['y1'], label['x1'] + label['w'], label['y1'] + label['h']] for label in labels],
        dtype=np.float64)
    scores = mx.nd.array(np.array([np.ones(bboxes.shape[0])[:, None]]))
    det_labels = mx.nd.array(np.array([label['person_id'] for label in labels]))
    colors = {label['person_id']: cg.id_to_random_color(seq.cluster_labels[idx], normalized=True) for idx, label in
              enumerate(labels)}

    img = gluoncv.utils.viz.cv_plot_bbox(img, bboxes, scores[0], labels=det_labels, colors=colors)
    cv2.imshow('frame', img)
    cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

  0%|          | 0/31 [00:00<?, ?it/s]QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToThread: Current thread (0x14131e50) is not the object's thread (0x20748ae0).
Cannot move to target thread (0x14131e50)

QObject::moveToTh

-1